# LLM standard library

Given our diagram's ability to use generic functions, we can create a standard library of functions that are useful for LLMs.

In [ ]:
# |default_exp chat

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

import pytest

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export
from stringdale.core import get_git_root, load_env, checkLogs,  json_render,json_undeclared_vars,disk_cache
import openai 
from pydantic import BaseModel, create_model
from typing import Optional, Dict, Any, List, Union
import json
import re
from parse import parse
from pathlib import Path
from enum import Enum
from pydantic import BaseModel
import logging
import os
from stringdale.core import semaphore_decorator


In [ ]:
#| export
load_env()
logger = logging.getLogger(__name__)


In [ ]:
#| export
from typing import (
    Callable, 
    Type, 
    Optional, 
    List, 
    Dict, 
    Any, 
    Union,
    Literal,
    get_type_hints
)
from pydantic import (
    BaseModel, 
    Field, 
    create_model,
    ConfigDict
)
import inspect
import sys
import sqlmodel
from typing import Optional
# TODO make chat outputschema also get typing (like bool)

In [ ]:
import nest_asyncio

In [ ]:
nest_asyncio.apply()

In [ ]:
#| export
import instructor
from openai import OpenAI,AsyncOpenAI
from anthropic import Anthropic, AsyncAnthropic
from pydantic import BaseModel
from singleton_decorator import singleton
import asyncio


## Completion functions

### OpenAI

In [ ]:
#| export
@singleton
def tools_openai_client():
    return instructor.from_openai(AsyncOpenAI(), mode=instructor.Mode.TOOLS)

@singleton
def json_openai_client():
    return instructor.from_openai(AsyncOpenAI(), mode=instructor.Mode.JSON)

@singleton
def raw_openai_client():
    return AsyncOpenAI()


In [ ]:
#| export
# Provider-specific completion function for open ai
@disk_cache.cache(ignore=['response_model'])
async def complete_open_ai(model, messages, response_model=None, response_schema=None, mode='json', seed=42, **kwargs):
    """
    OpenAI-specific completion handler.
    Chooses between clients based on mode and runs the completion.
    """
    if mode == 'json':
        client = json_openai_client()
    elif mode == 'tools':
        client = tools_openai_client()
    elif mode == 'raw':
        client = raw_openai_client()
        # For raw mode, we use the standard OpenAI client API
        completion = await client.chat.completions.create(
            model=model,
            messages=messages,
            seed=seed,
            **kwargs
        )
        usage = {
            "input_tokens": completion.usage.prompt_tokens,
            "output_tokens": completion.usage.completion_tokens
        }
        # Return the raw response content and usage
        return completion.choices[0].message.content, usage
    else:
        raise ValueError(f"Invalid mode: {mode}")
    
    # For json and tools modes (using instructor)
    response, completion = await client.chat.completions.create_with_completion(
        model=model,
        messages=messages,
        response_model=response_model,
        seed=seed,
        **kwargs
    )
    usage = {
        "input_tokens": completion.usage.prompt_tokens,
        "output_tokens": completion.usage.completion_tokens
    }
    return response.model_dump_json(), usage

### Anthropic

In [ ]:
## Anthropic clients
@singleton
def tools_anthropic_client():
    return instructor.from_anthropic(
        AsyncAnthropic(),
        mode=instructor.Mode.ANTHROPIC_TOOLS
    )

@singleton
def raw_anthropic_client():
    return AsyncAnthropic()
    
# Patching the Anthropics client with the instructor for enhanced capabilities
@singleton
def json_anthropic_client():
    return instructor.from_anthropic(
        AsyncAnthropic(),
        mode=instructor.Mode.ANTHROPIC_JSON
    )

In [ ]:
#| export
@disk_cache.cache(ignore=['response_model'])
async def complete_anthropic(model, messages, response_model=None, response_schema=None, mode='json', seed=42, **kwargs):
    """
    Anthropic-specific completion handler.
    Chooses between clients based on mode and runs the completion.
    """
    def _extract_usage(usage_obj):
        """Extract usage information from Anthropic usage object"""
        return {
            "input_tokens": getattr(usage_obj, "input_tokens", 0),
            "output_tokens": getattr(usage_obj, "output_tokens", 0)
        }
    
    max_tokens = kwargs.get('max_tokens', 1024)
    # Filter out unsupported Anthropic parameters #
    # Anthropic doesn't support 'stop' or 'seed' parameters
    anthropic_kwargs = {k: v for k, v in kwargs.items() 
                        if k not in ['stop', 'seed', 'print_prompt']}
    
    if mode == 'json':
        client = json_anthropic_client()
        response, completion = await client.chat.completions.create_with_completion(
            model=model,
            messages=messages,
            response_model=response_model,
            max_tokens=max_tokens,
            **anthropic_kwargs
        )
        return response.model_dump_json(), _extract_usage(completion.usage)

    elif mode == 'raw':
        client = raw_anthropic_client()
        
        # Extract system message (first system message), rest as regular messages
        system = None
        stripped = []
        for m in messages:
            role = m.get("role")
            content = m.get("content", "")
            if role == "system" and system is None:
                system = content
            else:
                stripped.append({"role": role, "content": content})
        
        # Call Anthropic API
        resp = await client.messages.create(
            model=model,
            system=system if system else "",  # Anthropic allows empty string for system
            messages=stripped,
            max_tokens=max_tokens,
            **anthropic_kwargs
        )
        
        # Aggregate text blocks from response
        parts = []
        for block in (resp.content or []):
            if getattr(block, "type", None) == "text":
                parts.append(block.text)
        content = "\n".join(parts)
        
        return content, _extract_usage(resp.usage)
    else:
        raise ValueError(f"Invalid mode: {mode}")

### Unified Chat Completion Wrapper

In [ ]:
#| export
async def complete_raw(model, messages, response_model=None, response_schema=None, mode='json', seed=42, provider=None, **kwargs):
    """
    This function is used to complete a chat completion with instructor without having basemodels as input or output.
    used for disk caching of results.
    
    Chooses what provider and calls a provider specific caller by model name with provider override.
    
    Args:
        model: Model name (e.g., 'gpt-4', 'claude-3-haiku-20240307')
        messages: List of message dicts
        response_model: Pydantic model for structured output
        response_schema: Schema for response (computed if response_model is provided)
        mode: Completion mode ('raw', 'json', 'tools')
        seed: Random seed
        provider: Optional provider override ('openai', 'anthropic'). If None, inferred from model name.
        **kwargs: Additional arguments passed to the provider.
                  Important provider-specific requirements:
                  - Anthropic: max_tokens is REQUIRED (e.g., max_tokens=1024)
    """
    # Determine provider from model name or use override
    if provider is None:
        if model.startswith(('gpt-', 'o1-')):
            provider = 'openai'
        elif model.startswith('claude-'):
            provider = 'anthropic'
        else:
            # Default to OpenAI for unknown models
            provider = 'openai'
    
    # Map provider names to their completion functions
    provider_functions = {
        'openai': complete_open_ai,
        'anthropic': complete_anthropic
    }

    complete_func = provider_functions.get(provider)
    if complete_func is None:
        raise ValueError(f"Unknown provider: {provider}")

    return await complete_func(
        model=model,
        messages=messages,
        response_model=response_model,
        response_schema=response_schema,
        mode=mode,
        seed=seed,
        **kwargs
    )

async def complete(model, messages, response_model, mode='json', print_prompt=False, **kwargs):
    # Compute schema if response model provided
    if isinstance(response_model, type) and issubclass(response_model, BaseModel):
        response_schema = response_model.model_json_schema()
    else:
        response_schema = str(response_model)
    if print_prompt:
        print(messages)
    response, usage = await complete_raw(
        model=model,
        messages=messages,
        response_model=response_model,
        response_schema=response_schema,
        mode=mode,
        **kwargs
    )
    if mode == 'raw':
        return response, usage
    else:
        # Check if response is already a model instance (from instructor)
        if isinstance(response, BaseModel):
            return response, usage
        else:
            return response_model.model_validate_json(response), usage

In [ ]:
class UserExtract(BaseModel):
    name: str
    age: int


user, usage = await complete(
    model="gpt-3.5-turbo",
    response_model=UserExtract,
    messages=[
        {"role": "user", "content": "Extract jason is 25 years old"},
    ],
)

user,usage

(UserExtract(name='Extract jason', age=25),
 {'input_tokens': 147, 'output_tokens': 18})

In [ ]:
user, usage = await complete(
    model="claude-sonnet-4-5",
    response_model=UserExtract,
    messages=[
        {"role": "user", "content": "Extract jason is 25 years old"},
    ],
)

user,usage

(UserExtract(name='jason', age=25), {'input_tokens': 163, 'output_tokens': 39})

## Completion types

### Simpler answer

In [ ]:
#| export
async def answer_question(model,messages,**api_kwargs):
    res,usage = await complete(model,messages,response_model=None,mode='raw',**api_kwargs)
    return res,usage


In [ ]:
answer = await answer_question("gpt-3.5-turbo",[{"role":"user","content":"What is the capital of France?"}])
print(answer)

('The capital of France is Paris.', {'input_tokens': 14, 'output_tokens': 7})


In [ ]:
# Test: Integration test with answer_question - anthropic
answer, usage = await answer_question(
    model="claude-sonnet-4-5",
    messages=[{"role": "user", "content": "What is the capital of France?"}],
)

assert isinstance(answer, str)
assert "Paris" in answer or "paris" in answer.lower()
assert isinstance(usage, dict)

### Choice

In [ ]:
#| export
async def choose(model,messages,choices,**api_kwargs):
    class Choice(BaseModel):
        choice: Literal[tuple(choices)]
    res,usage = await complete(model,messages,Choice,**api_kwargs)
    return res.choice,usage


In [ ]:
await choose("gpt-3.5-turbo",[{"role":"user","content":"What is the capital of the country France?"}],["PARIS", "HILTON"],print_prompt=True)


[{'role': 'user', 'content': 'What is the capital of the country France?'}]


('PARIS', {'input_tokens': 140, 'output_tokens': 10})

In [ ]:
await choose("claude-sonnet-4-5",[{"role":"user","content":"What is the capital of the country France?"}],["PARIS", "HILTON"],print_prompt=True)

[{'role': 'user', 'content': 'What is the capital of the country France?'}]


('PARIS', {'input_tokens': 154, 'output_tokens': 20})

### Multi choice

In [ ]:
#| export
async def choose_many(model,messages,choices,**api_kwargs):
    class Choice(BaseModel):
        choice: Literal[tuple(choices)]

    class Choices(BaseModel):
        choices: List[Choice]   
    res,usage = await complete(model,messages,Choices,**api_kwargs)
    return [c.choice for c in res.choices],usage


In [ ]:

new_answer = await choose_many("gpt-3.5-turbo",
    messages=[{"role":"user","content":"what parameters did i pass in? my name is jason and i am 25 years old"}],
    choices=["Age", "Name","City"],print_prompt=True)

print(new_answer)

[{'role': 'user', 'content': 'what parameters did i pass in? my name is jason and i am 25 years old'}]
(['Name', 'Age'], {'input_tokens': 235, 'output_tokens': 31})


In [ ]:
# calude
await choose_many("claude-sonnet-4-5",
    messages=[{"role":"user","content":"what parameters did i pass in? my name is jason and i am 25 years old"}],
    choices=["Age", "Name","City"],print_prompt=True)

[{'role': 'user', 'content': 'what parameters did i pass in? my name is jason and i am 25 years old'}]


(['Name', 'Age'], {'input_tokens': 272, 'output_tokens': 101})

### Structured output

In [ ]:
#| export
def clean_model(model: Type[sqlmodel.SQLModel], name: Optional[str] = None) -> Type[BaseModel]:
    """Convert an SQLModel to a Pydantic BaseModel.
    used to clean up the output for the LLM
    Args:
        model: SQLModel class to convert
        name: Optional name for the new model class
        
    Returns:
        A Pydantic BaseModel class with the same fields
    """
    # Get field definitions from the SQLModel
    fields = {}
    for field_name, field in model.model_fields.items():
        fields[field_name] = (field.annotation, field)
    
    # Create new model name if not provided
    model_name = name or f"{model.__name__}Schema"
    
    # Create and return new Pydantic model
    return create_model(model_name, **fields)

In [ ]:
#| export
async def structured_output(model,messages,output_schema,as_json=False,**api_kwargs):

    is_sqlmodel = isinstance(output_schema,type) and issubclass(output_schema,sqlmodel.SQLModel)
    if is_sqlmodel:
        clean_schema = clean_model(output_schema)
    else:
        clean_schema = output_schema

    res,usage = await complete(model,messages,clean_schema,**api_kwargs)

    if is_sqlmodel:
        res = output_schema(**res.model_dump())
    if as_json:
        res = res.model_dump()
    return res,usage


In [ ]:
class UserExtract(BaseModel):
    name: str
    age: int



res,usage = await structured_output("gpt-3.5-turbo",
    [{"role":"user","content":"what parameters did i pass in? my name is Jason and i am 25 years old"}],
    UserExtract)
assert res == UserExtract(name="Jason",age=25), res
print(res)
print(usage)




name='Jason' age=25
{'input_tokens': 157, 'output_tokens': 16}


In [ ]:
# claude
res,usage = await structured_output("claude-sonnet-4-5",
    [{"role":"user","content":"what parameters did i pass in? my name is Jason and i am 25 years old"}],
    UserExtract)

print(res)
print(usage)



name='Jason' age=25
{'input_tokens': 174, 'output_tokens': 72}


In [ ]:

import sqlite3
from sqlalchemy.engine import create_engine

In [ ]:

#| export
class User(sqlmodel.SQLModel, table=False):
    id: Optional[int] = sqlmodel.Field(default=None, primary_key=True)
    name: Optional[str] = sqlmodel.Field(default=None)
    age: Optional[int] = sqlmodel.Field(default=None)
    email: Optional[str] = sqlmodel.Field(default=None)



In [ ]:

res,usage = await structured_output("gpt-3.5-turbo",
    [{"role":"user","content":"my name is jyson, my age is 25, my id is 1"}],
    User)
print(res)


id=1 name='jyson' age=25 email=None


In [ ]:
# anthropic
anth_res,usage = await structured_output("claude-sonnet-4-5",
    [{"role":"user","content":"my name is jyson, my age is 25, my id is 1"}],
    User)
print(res)
assert anth_res==res

id=1 name='jyson' age=25 email=None


### Tool calling

In [ ]:
#| export
import docstring_parser 


In [ ]:
#| export
def function_to_input_description(func: Callable) -> Dict[str, Any]:
    """Extract parameter information from a function's signature and docstring.
    
    Args:
        func: Function to analyze
        
    Returns:
        Dictionary containing:
            - name: Function name
            - params: Dict of parameter info with:
                - type: Parameter type annotation
                - description: Parameter description from docstring
                - default: Default value if any
    """
    # Get type hints and signature
    hints = get_type_hints(func)
    sig = inspect.signature(func)
    
    # Remove return annotation if present
    hints.pop('return', None)
    
    # Parse docstring for parameter descriptions
    docstring = inspect.getdoc(func)
    arg_descriptions = {}
    if docstring:
        parsed = docstring_parser.parse(docstring)
        arg_descriptions = {
            p.arg_name: p.description 
            for p in parsed.params
        }
    
    # Build parameter info
    params = {}
    for name, param in sig.parameters.items():
        params[name] = {
            'type': hints.get(name, Any),
            'description': arg_descriptions.get(name, f"Parameter {name}"),
            'default': None if param.default == inspect.Parameter.empty else param.default
        }
    
    return {
        'name': func.__name__,
        'params': params
    }

def description_to_model(desc: Dict[str, Any], model_name: Optional[str] = None) -> Type[BaseModel]:
    """Create a Pydantic model from a function description.
    
    Args:
        desc: Function description from function_to_input_description
        model_name: Optional name for the model class
        
    Returns:
        Pydantic model class
    """
    # Create model fields
    fields = {}
    
    for name, info in desc['params'].items():
        field = Field(
            description=info['description'],
            default=info['default'] if info['default'] is not None else ...
        )
        fields[name] = (info['type'], field)
    
    # Create model class
    model_name = model_name or f"{desc['name']}Input"
    return create_model(
        model_name, 
        __config__=ConfigDict(extra='forbid'),
        **fields
    )



In [ ]:
async def add(a: int, b: int = 0) -> int:
    """Add two numbers.
    
    Args:
        a: First number to add
        b: Second number to add
    """
    return a + b

# Get function description
desc = function_to_input_description(add)
assert desc == {
    'name': 'add',
    'params': {'a': {'type': int,'description': 'First number to add','default': None},
               'b': {'type': int, 'description': 'Second number to add', 'default': 0}
               }
    }

In [ ]:
def math_op(op: Literal["add", "multiply", "divide"],a: int=0, b: int = 0) -> int:
    """Add two numbers.
    
    Args:
        op: Operation to perform
        a: First number to add
        b: Second number to add
    """
    return a + b

# Get function description
desc = function_to_input_description(math_op)
desc
assert desc == {'name': 'math_op',
 'params': {'op': {'type': Literal['add', 'multiply', 'divide'],
   'description': 'Operation to perform',
   'default': None},
  'a': {'type': int, 'description': 'First number to add', 'default': 0},
  'b': {'type': int, 'description': 'Second number to add', 'default': 0}}}

In [ ]:
#| export
def function_to_input_model(func: Callable,name:str,descriminator_field:str="tool_name") -> Type[BaseModel]:
    """Convert a function to a Pydantic input model.
    
    Args:
        func: Function to analyze
        
    Returns:
        Pydantic model class for function inputs
    """
    desc = function_to_input_description(func)
    desc['params'][descriminator_field] = {
        'type': Literal[name],
        'description': 'The name of the function to call',
        'default': None
    }
    return description_to_model(desc)


In [ ]:
#| export
async def call_tools(
    model: str,
    messages: List[Dict[str, str]], 
    tools: Dict[str, Callable],
    call_function: bool=False,
    descriminator_field:str="tool_name",
    **api_kwargs
) -> Dict[str, Any]:
    """Call OpenAI chat completion with tool selection and input parsing.
    
    Args:
        model: OpenAI model name
        messages: List of message dicts with role and content
        tools: Dictionary mapping tool names to functions
        descriminator_field: The name of the field to use as the discriminator
    Returns:
        Dict with:
            - tool_name: Selected tool name
            - tool_input: Parsed input for the tool
    """
    # Create input models for each tool
    tool_models = {
        name: function_to_input_model(func,name,descriminator_field)
        for name, func in tools.items()
    }
    
    # Create discriminated union model for tool inputs
    class ToolsInput(BaseModel):
        tool_input: Union[tuple(tool_models.values())] = Field(discriminator=descriminator_field)
    
    tool_description = '\n'.join([f'{name}:{inspect.getdoc(func)}' for name,func in tools.items()])
    system_message = [
        {'role':'system',
        'content':f'choose an appropriate tool to use to answer the following thought based on the following tools:\n'
                  f'{tool_description}'
        }
    ]
    messages = system_message + messages

    # Get tool input from LLM
    response, usage = await complete(
        model=model,
        messages=messages,
        response_model=ToolsInput,
        mode='tools',
        **api_kwargs
    )
    
    kwargs = response.tool_input.model_dump()
    func_name = kwargs.pop(descriminator_field)

    result = {
        'name': func_name,
        'input': kwargs,
    }
    if call_function:
        try:
            result['output'] = tools[func_name](**kwargs)
        except Exception as e:
            result['output'] = f'Error: {str(e)}'
    return result,usage
    

In [ ]:
def add(a: int, b: int = 0) -> int:
    """Add two numbers.
    
    Args:
        a: First number to add
        b: Second number to add
    """
    return a + b

def multiply(x: int, y: int) -> int:
    """Multiply two numbers.
    
    Args:
        x: First number to multiply
        y: Second number to multiply
    """
    return x * y

def divide(numerator: int, denominator: int) -> float:
    """Divide two numbers.
    
    Args:
        numerator: Number to divide
        denominator: Number to divide by
    """
    return numerator / denominator

tools = {
    'add': add,
    'multiply': multiply,
    'divide': divide
}


In [ ]:

result,usage = await call_tools(
    model="gpt-3.5-turbo",  
    messages=[{"role": "user", "content": "What is 5 plus 3?"}],
    tools=tools,
    print_prompt=True
)
assert result == {'name': 'add', 'input': {'a': 5, 'b': 3}}
result,usage

[{'role': 'system', 'content': 'choose an appropriate tool to use to answer the following thought based on the following tools:\nadd:Add two numbers.\n\nArgs:\n    a: First number to add\n    b: Second number to add\nmultiply:Multiply two numbers.\n\nArgs:\n    x: First number to multiply\n    y: Second number to multiply\ndivide:Divide two numbers.\n\nArgs:\n    numerator: Number to divide\n    denominator: Number to divide by'}, {'role': 'user', 'content': 'What is 5 plus 3?'}]


({'name': 'add', 'input': {'a': 5, 'b': 3}},
 {'input_tokens': 335, 'output_tokens': 17})

In [ ]:
result,usage = await call_tools(
    model="gpt-3.5-turbo",  
    messages=[{"role": "user", "content": "What is 5 plus 3?"}],
    tools=tools,
    call_function=True
)
assert result == {'name': 'add', 'input': {'a': 5, 'b': 3}, 'output': 8}
result,usage

({'name': 'add', 'input': {'a': 5, 'b': 3}, 'output': 8},
 {'input_tokens': 335, 'output_tokens': 17})

In [ ]:
def exception_raiser(a:int,b:int):
    raise ValueError("This is a test error")

tools= {
    'add': exception_raiser,
}

result,usage = await call_tools(
    model="gpt-3.5-turbo",  
    messages=[{"role": "user", "content": "What is 5 plus 3?"}],
    tools=tools,
    call_function=True
    )

assert result == {'name': 'add', 'input': {'a': 5, 'b': 3}, 'output': 'Error: This is a test error'},result
result,usage

({'name': 'add',
  'input': {'a': 5, 'b': 3},
  'output': 'Error: This is a test error'},
 {'input_tokens': 149, 'output_tokens': 17})

## The main chat class

In [ ]:
#| export
from copy import deepcopy,copy
from pprint import pformat,pprint


In [ ]:
#| export

class Chat:
    """A Chat objects the renders a prompt and calls an LLM. Currently supporting openai models.
    
    Args:
        model: OpenAI model name
        messages: List of message dicts, must have at least a role and content field
        output_schema: Optional schema for structured output
        as_json: Optional boolean to return the response as a json object
        tools: Optional dictionary of tool names and functions that the LLM can decide to call. Causes the content of the response
            to be a dict of the form {'name':tool_name,'input':tool_input_dict}
        call_function: if tools are provided, whether to call the function and save the output in the output field of the response's content
        choices: Optional List of choices for multi-choice questions
        multi_choice: if choices are provided, whether to choose multiple items from the list
        seed: Optional seed for random number generation
        stop: Optional string or list of strings where the model should stop generating
        save_history: Optional boolean to save the history of the chat between calls
        append_output: Optional, whether to append the output of the chat to history automatically, default False
        init_messages: Optional list of messages that are always prepended to messages.
            Useful for supplying additional messages during calls.
            Can have template variables that are fed during initialization only.
            If save_history is True, the init messages are added to the history.
        **kwargs: Keyword arguments to interpolate into the messages
    """
    def __init__(self,
        model: Optional[str] = None,
        messages: Optional[List[Dict[str, str]]] = None, 
        output_schema: Optional[BaseModel] = None,
        as_json: Optional[bool] = False,
        tools: Optional[Dict[str,Callable]] = None,
        call_function: Optional[bool] = False,
        choices: Optional[Enum] = None,
        multi_choice: Optional[bool] = False,
        seed: Optional[int] = 42,
        stop: Optional[Union[str, List[str]]] = None,
        log_prompt: bool = False,
        save_history: bool = False,
        append_output: bool = False,
        init_messages: Optional[List[Dict[str, str]]] = None,
        **kwargs):

        self.model = model
        self.messages = deepcopy(messages)
        self.output_schema = output_schema
        self.as_json = as_json
        self.tools = tools
        self.call_function = call_function
        self.choices = choices
        self.multi_choice = multi_choice
        self.seed = seed
        self.stop = stop
        self.log_prompt = log_prompt
        self.baked_kwargs = kwargs
        self.save_history = save_history
        self.append_output = append_output
        
        if init_messages is None:
            init_messages = []
        self.init_messages = json_render(init_messages,context=kwargs)
    
        self.reset()

    def reset(self):
        """Resets state of Chat"""
        if self.save_history:
            self.history = []
            self.history.extend(self.init_messages)
        else:
            self.history = None
    
    def dump_state(self):
        """dumps the node state"""
        return self.history

    def load_state(self,state_object):
        """loads node state"""
        self.history = state_object

    def __copy__(self):
        chat_copy = Chat(
            model=self.model,
            messages=self.messages,
            output_schema=self.output_schema,
            as_json=self.as_json,
            tools=self.tools,
            call_function=self.call_function,
            choices=self.choices,
            multi_choice=self.multi_choice,
            seed=self.seed,
            stop=self.stop,
            log_prompt=self.log_prompt,
            save_history=self.save_history,
            append_output=self.append_output,
            init_messages=self.init_messages,
            **self.baked_kwargs
        )
        return chat_copy

    async def __call__(self, **kwargs) -> Dict[str, Any]:
        """Format prompt with kwargs and call OpenAI chat.
        Init parameters such as output_schema, tools, choices, seed, stop, as well as template variables
        can be set or overridden by kwargs
        
        Args:
            **kwargs: Values for format string placeholders
            
        Returns:
            a dictionary with the following keys:
            - role (str): Always "assistant"
            - content: the llm response.
            - meta (dict): Usage statistics including input and output tokens
        """
        model = kwargs.get("model", self.model)
        messages = kwargs.get("messages", self.messages)
        output_schema = kwargs.get("output_schema", self.output_schema)
        as_json = kwargs.get("as_json", self.as_json)
        tools = kwargs.get("tools", self.tools)
        call_function = kwargs.get("call_function", self.call_function)
        choices = kwargs.get("choices", self.choices)
        multi_choice = kwargs.get("multi_choice", self.multi_choice)
        seed = kwargs.get("seed", self.seed)
        stop = kwargs.get("stop", self.stop)

        if model is None:
            raise ValueError("model is required but not provided")
        if messages is None:
            raise ValueError("messages is required but not provided")

        prompt_kwargs = {**self.baked_kwargs, **kwargs}

        required_kwargs = json_undeclared_vars(messages)
        if not required_kwargs <= set(prompt_kwargs):
            missing = required_kwargs - set(prompt_kwargs)
            raise ValueError(f"Missing required kwargs: {missing}")

        formatted_messages = json_render(messages, context=prompt_kwargs)

        if self.save_history:
            self.history.extend(formatted_messages)
            formatted_messages = self.history
        else:
            formatted_messages = self.init_messages + formatted_messages

        if self.log_prompt:
            logger.warning(f'calling llm with model={model} and prompt:\n'
                        f'messages={pformat(formatted_messages)}\n'
                        )

        completion_kwargs = {
            'model':model,
            'messages':formatted_messages,
            'seed':seed,
            'stop':stop,
            'print_prompt':prompt_kwargs.get('print_prompt',False),
        }

        if choices:
            if multi_choice:
                res,usage = await choose_many(choices=choices,**completion_kwargs)
            else:
                res,usage = await choose(choices=choices,**completion_kwargs)
        elif output_schema:
            res,usage = await structured_output(output_schema=output_schema,as_json=as_json,**completion_kwargs)
        elif tools:
            res,usage = await call_tools(tools=tools,call_function=call_function,**completion_kwargs)
        else:
            res,usage = await answer_question(**completion_kwargs)

        response = {
            'role':'assistant',
            'content':res,
            'meta':usage
        }
        if self.save_history and self.append_output:
            self.history.append(response)
        return response
    
    def __str__(self) -> str:
        """String representation showing required keys, model, and output schema."""
        parts = [f"Chat(model='{self.model}'"]
        
        if self.messages:
            required_keys = json_undeclared_vars(self.messages) - set(self.baked_kwargs.keys())
            parts.append(f"required_keys={required_keys}")
            
        if self.output_schema:
            parts.append(f"output_schema={self.output_schema.__name__}")
        
        if self.tools:
            parts.append(f"""tools={",".join(self.tools.keys())}""")
        if self.call_function:
            parts.append(f"call_function={self.call_function}")
            
        # if self.choices:
        #     parts.append(f"choices={self.choices}")
        # if self.multi_choice:
        #     parts.append(f"multi_choice={self.multi_choice}")
            
        if self.seed:
            parts.append(f"seed={self.seed}")
            
        if self.stop:
            parts.append(f"stop={self.stop}")

        if self.save_history:
            parts.append(f"save_history={self.save_history}")
            
        return ", ".join(parts) + ")"
    
    def metadata(self) -> Dict[str, Any]:
        """Return metadata about the chat."""
        meta =  {
            'model':self.model,
            'messages':self.messages,
            'output_schema':self.output_schema,
            'tools':self.tools,
            'call_function':self.call_function,
            'choices':self.choices,
            'multi_choice':self.multi_choice,
            'seed':self.seed,
            'stop':self.stop,
            
        }
        if self.save_history:
            meta['history'] = self.history
        meta = {k:v for k,v in meta.items() if v is not None and v is not False}
        return meta

    def __repr__(self) -> str:
        """Same as string representation."""
        return self.__str__()

For a cheatsheet on how to use jinja templates, see [this link](https://devhints.io/jinja)

## Chat Tests

### Basic

In [ ]:
chat_with_history = Chat(model="claude-sonnet-4-5", 
                        save_history=True,
                        append_output=True,
                        init_messages=[{"role":"system","content":"You are a helpful {{role}}"}],
                        messages=[{"role": "user", "content": "Hi, im {{name}}, answer me: {{text}}"}],
                        role = 'AI overlord',
                        name = 'ernio',
                        log_prompt=True
                        )
res = await chat_with_history(text="What is the capital of France?",print_prompt=True)
res

calling llm with model=claude-sonnet-4-5 and prompt:
messages=[{'content': 'You are a helpful AI overlord', 'role': 'system'},
 {'content': 'Hi, im ernio, answer me: What is the capital of France?',
  'role': 'user'}]



[{'role': 'system', 'content': 'You are a helpful AI overlord'}, {'role': 'user', 'content': 'Hi, im ernio, answer me: What is the capital of France?'}]


{'role': 'assistant',
 'content': 'Hello Ernio!\n\nThe capital of France is **Paris**.',
 'meta': {'input_tokens': 30, 'output_tokens': 17}}

In [ ]:
chat_with_history = Chat(model="gpt-3.5-turbo", 
                        save_history=True,
                        append_output=True,
                        init_messages=[{"role":"system","content":"You are a helpful {{role}}"}],
                        messages=[{"role": "user", "content": "Hi, im {{name}}, answer me: {{text}}"}],
                        role = 'AI overlord',
                        name = 'ernio',
                        log_prompt=True
                        )
res = await chat_with_history(text="What is the capital of France?",print_prompt=True)
res

calling llm with model=gpt-3.5-turbo and prompt:
messages=[{'content': 'You are a helpful AI overlord', 'role': 'system'},
 {'content': 'Hi, im ernio, answer me: What is the capital of France?',
  'role': 'user'}]



[{'role': 'system', 'content': 'You are a helpful AI overlord'}, {'role': 'user', 'content': 'Hi, im ernio, answer me: What is the capital of France?'}]


{'role': 'assistant',
 'content': 'Hello Ernio! The capital of France is Paris.',
 'meta': {'input_tokens': 34, 'output_tokens': 11}}

In [ ]:
chat_with_history.history

[{'role': 'system', 'content': 'You are a helpful AI overlord'},
 {'role': 'user',
  'content': 'Hi, im ernio, answer me: What is the capital of France?'},
 {'role': 'assistant',
  'content': 'Hello Ernio! The capital of France is Paris.',
  'meta': {'input_tokens': 34, 'output_tokens': 11}}]

In [ ]:
for i in range(3):
    res = await chat_with_history(messages=[{'role':'user','content':'Question: what is obamas age to the power of 2?'}])
    chat_with_history.reset()
    # print(chat_with_history.history)
    print(res)
    print('='*100)

calling llm with model=gpt-3.5-turbo and prompt:
messages=[{'content': 'You are a helpful AI overlord', 'role': 'system'},
 {'content': 'Hi, im ernio, answer me: What is the capital of France?',
  'role': 'user'},
 {'content': 'Hello Ernio! The capital of France is Paris.',
  'meta': {'input_tokens': 34, 'output_tokens': 11},
  'role': 'assistant'},
 {'content': 'Question: what is obamas age to the power of 2?', 'role': 'user'}]

calling llm with model=gpt-3.5-turbo and prompt:
messages=[{'content': 'You are a helpful AI overlord', 'role': 'system'},
 {'content': 'Question: what is obamas age to the power of 2?', 'role': 'user'}]

calling llm with model=gpt-3.5-turbo and prompt:
messages=[{'content': 'You are a helpful AI overlord', 'role': 'system'},
 {'content': 'Question: what is obamas age to the power of 2?', 'role': 'user'}]



{'role': 'assistant', 'content': "To calculate President Obama's age to the power of 2, we first need to know his current age. As of 2021, Barack Obama was born on August 4, 1961. Therefore, if we assume it's 2021, his age would be 60.\n\nNow let's calculate his age to the power of 2:\n\nAge squared = 60^2 = 60 x 60 = 3600\n\nSo, President Obama's age to the power of 2 is 3600.", 'meta': {'input_tokens': 67, 'output_tokens': 107}}
{'role': 'assistant', 'content': "Obama was born on August 4, 1961. To find his current age squared, we first need to find his age by subtracting his birth year from the current year. \nCurrent year - Birth year = Age\n2023 - 1961 = 62\n\nNow, we can calculate Obama's age to the power of 2:\n62^2 = 62 x 62 = 3844\n\nTherefore, Obama's age to the power of 2 is 3844.", 'meta': {'input_tokens': 32, 'output_tokens': 101}}
{'role': 'assistant', 'content': "Obama was born on August 4, 1961. To find his current age squared, we first need to find his age by subtracti

In [ ]:
assert len(chat_with_history.history) == 1 , len(chat_with_history.history)
chat_with_history.history

[{'role': 'system', 'content': 'You are a helpful AI overlord'}]

In [ ]:
res = await chat_with_history(text="And what is the closest city to it?",print_prompt=True)
res

calling llm with model=gpt-3.5-turbo and prompt:
messages=[{'content': 'You are a helpful AI overlord', 'role': 'system'},
 {'content': 'Hi, im ernio, answer me: And what is the closest city to it?',
  'role': 'user'}]



[{'role': 'system', 'content': 'You are a helpful AI overlord'}, {'role': 'user', 'content': 'Hi, im ernio, answer me: And what is the closest city to it?'}]


{'role': 'assistant',
 'content': "Hello, Ernio. I'm here to help. Could you please provide more context or clarify your question about the nearest city? I'd be happy to assist you once I have a better understanding of what you're looking for.",
 'meta': {'input_tokens': 36, 'output_tokens': 46}}

In [ ]:
assert len(chat_with_history.history)==3

### Choices

In [ ]:
messages=[
        {"role": "system", "content": "Given a sentence, classify it into one of these topics: science, history, technology, or arts. Choose the single most relevant topic."},
        {"role": "user", "content": "{{text}}"}
    ]
    
topic_classifier = Chat(
    model="gpt-4o-mini",
    messages=messages,
    choices = ['science', 'history', 'technology', 'arts'],
    seed=42,
    log_prompt=True
)

topic_classifier_anthropic = Chat(
    model="claude-sonnet-4-5",
    messages=messages,
    choices = ['science', 'history', 'technology', 'arts'],
    seed=42,
    log_prompt=True
)


In [ ]:
renaisance_topic = await topic_classifier(text="WWII was a global conflict that lasted from 1939 to 1945.")
astroid_topic = await topic_classifier(text="The asteroid belt is a region of space between the orbits of Mars and Jupiter.")

assert renaisance_topic['content'] == 'history',renaisance_topic
assert astroid_topic['content'] == 'science',astroid_topic



calling llm with model=gpt-4o-mini and prompt:
messages=[{'content': 'Given a sentence, classify it into one of these topics: science, '
             'history, technology, or arts. Choose the single most relevant '
             'topic.',
  'role': 'system'},
 {'content': 'WWII was a global conflict that lasted from 1939 to 1945.',
  'role': 'user'}]

calling llm with model=gpt-4o-mini and prompt:
messages=[{'content': 'Given a sentence, classify it into one of these topics: science, '
             'history, technology, or arts. Choose the single most relevant '
             'topic.',
  'role': 'system'},
 {'content': 'The asteroid belt is a region of space between the orbits of '
             'Mars and Jupiter.',
  'role': 'user'}]



In [ ]:
renaisance_topic = await topic_classifier_anthropic(text="WWII was a global conflict that lasted from 1939 to 1945.")
astroid_topic = await topic_classifier_anthropic(text="The asteroid belt is a region of space between the orbits of Mars and Jupiter.")

assert renaisance_topic['content'] == 'history',renaisance_topic
assert astroid_topic['content'] == 'science',astroid_topic



calling llm with model=claude-sonnet-4-5 and prompt:
messages=[{'content': 'Given a sentence, classify it into one of these topics: science, '
             'history, technology, or arts. Choose the single most relevant '
             'topic.',
  'role': 'system'},
 {'content': 'WWII was a global conflict that lasted from 1939 to 1945.',
  'role': 'user'}]

calling llm with model=claude-sonnet-4-5 and prompt:
messages=[{'content': 'Given a sentence, classify it into one of these topics: science, '
             'history, technology, or arts. Choose the single most relevant '
             'topic.',
  'role': 'system'},
 {'content': 'The asteroid belt is a region of space between the orbits of '
             'Mars and Jupiter.',
  'role': 'user'}]



### Structured output

In [ ]:
class Person(BaseModel):
    first_name: str
    last_name: str
    date_of_birth: int

prompted_llm = Chat(model="gpt-4o-mini", messages=
    [   
        {"role": "user", "content": "how old am i? {{name}}, {{age}} years old"},
        {"role": "assistant", "content": "Iam {{model_name}}, You are {{name}}, {{age}} years old"}
    ],
     output_schema=Person)
prompted_llm


Chat(model='gpt-4o-mini', required_keys={'age', 'name', 'model_name'}, output_schema=Person, seed=42)

In [ ]:
prompted_llm_anthropic = Chat(model="claude-sonnet-4-5", messages=
    [   
        {"role": "user", "content": "how old am i? {{name}}, {{age}} years old"},
        {"role": "assistant", "content": "Iam {{model_name}}, You are {{name}}, {{age}} years old"}
    ],
     output_schema=Person)
prompted_llm_anthropic

Chat(model='claude-sonnet-4-5', required_keys={'age', 'name', 'model_name'}, output_schema=Person, seed=42)

In [ ]:
baked_llm = Chat(model="gpt-4o-mini", messages=
    [
        {"role": "user", "content": "how old am i? {{name}}, {{age}} years old"},
        {"role": "assistant", "content": "Iam {{model_name}}, You are {{name}}, {{age}} years old"}
    ],
    output_schema=Person, model_name="gpt-4o-mini", age=30)
baked_llm


Chat(model='gpt-4o-mini', required_keys={'name'}, output_schema=Person, seed=42)

In [ ]:
baked_llm_anthropic = Chat(model="claude-sonnet-4-5", messages=
    [
        {"role": "user", "content": "how old am i? {{name}}, {{age}} years old"},
        {"role": "assistant", "content": "Iam {{model_name}}, You are {{name}}, {{age}} years old"}
    ],
    output_schema=Person, model_name="gpt-4o-mini", age=30)
baked_llm_anthropic

Chat(model='claude-sonnet-4-5', required_keys={'name'}, output_schema=Person, seed=42)

In [ ]:
res = await prompted_llm(model_name="gpt-4o-mini", age=30,name="Dean")
assert res['content'] == Person(first_name='Dean', last_name='', date_of_birth=1993) # huh?
res

{'role': 'assistant',
 'content': Person(first_name='Dean', last_name='', date_of_birth=1993),
 'meta': {'input_tokens': 206, 'output_tokens': 26}}

In [ ]:
res = await prompted_llm_anthropic(model_name="claude-sonnet-4-5", age=30,name="Dean")
assert res['content'].first_name == 'Dean'
assert res['content'].last_name == ''
assert 1990 <= res['content'].date_of_birth <= 2000  # Reasonable range for age 30
res

{'role': 'assistant',
 'content': Person(first_name='Dean', last_name='', date_of_birth=1995),
 'meta': {'input_tokens': 236, 'output_tokens': 189}}

In [ ]:
res = await baked_llm(name="Dean")
assert res['content'].first_name == 'Dean'
assert res['content'].last_name == ''
assert 1990 <= res['content'].date_of_birth <= 2000  # Reasonable range for age 30
res

{'role': 'assistant',
 'content': Person(first_name='Dean', last_name='', date_of_birth=1993),
 'meta': {'input_tokens': 206, 'output_tokens': 26}}

In [ ]:
res = await baked_llm_anthropic(name="Dean")
assert res['content'].first_name == 'Dean'
assert res['content'].last_name == ''
assert 1990 <= res['content'].date_of_birth <= 2000  # Reasonable range for age 30
res

{'role': 'assistant',
 'content': Person(first_name='Dean', last_name='', date_of_birth=1994),
 'meta': {'input_tokens': 235, 'output_tokens': 172}}

In [ ]:
res = await baked_llm(name="Dean",as_json=True)
assert res['content']['first_name'] == 'Dean'
assert res['content']['last_name'] == ''
assert 1990 <= res['content']['date_of_birth'] <= 2000  # Reasonable range for age 30
res

{'role': 'assistant',
 'content': {'first_name': 'Dean', 'last_name': '', 'date_of_birth': 1993},
 'meta': {'input_tokens': 206, 'output_tokens': 26}}

In [ ]:
res = await baked_llm_anthropic(name="Dean",as_json=True)
assert res['content']['first_name'] == 'Dean'
assert res['content']['last_name'] == ''
assert 1990 <= res['content']['date_of_birth'] <= 2000  # Reasonable range for age 30
res

{'role': 'assistant',
 'content': {'first_name': 'Dean', 'last_name': '', 'date_of_birth': 1994},
 'meta': {'input_tokens': 235, 'output_tokens': 172}}

In [ ]:
course_chooser = Chat(model="gpt-4o-mini",
    messages=[{"role": "user", "content": "{{text}}"}],
    choices=["science","genocide", "history", "defence against the dark arts", "arts"],
    multi_choice=True)
res = await course_chooser(text="choose everything that is not genocide")
assert not 'genocide' in res['content'] , res
res

{'role': 'assistant',
 'content': ['science', 'history', 'defence against the dark arts', 'arts'],
 'meta': {'input_tokens': 236, 'output_tokens': 58}}

In [ ]:
course_chooser_anthropic = Chat(model="claude-sonnet-4-5",
    messages=[{"role": "user", "content": "{{text}}"}],
    choices=["science","genocide", "history", "defence against the dark arts", "arts"],
    multi_choice=True)
res = await course_chooser(text="choose everything that is not genocide")
assert not 'genocide' in res['content'] , res
res

{'role': 'assistant',
 'content': ['science', 'history', 'defence against the dark arts', 'arts'],
 'meta': {'input_tokens': 236, 'output_tokens': 58}}

In [ ]:
def google_search_stub(query:str):
    """
    Search the web for the query
    Args:
        query: The query to search for
    Returns:
        The URL of the search results
    """
    return f"https://www.google.com/search?q={query.replace(' ','_')}"

tools = {'google_search': google_search_stub}

google_search = Chat(model="gpt-4o-mini", messages=[{"role": "user", "content": "{{text}}"}], tools=tools, call_function=True)
res = await google_search(text="What is the capital of France?")
assert res['content'] == {'name': 'google_search',
  'input': {'query': 'What is the capital of France?'},
  'output': 'https://www.google.com/search?q=What_is_the_capital_of_France?'}
res


{'role': 'assistant',
 'content': {'name': 'google_search',
  'input': {'query': 'What is the capital of France?'},
  'output': 'https://www.google.com/search?q=What_is_the_capital_of_France?'},
 'meta': {'input_tokens': 159, 'output_tokens': 20}}

### Init messages without saving history

In [ ]:
chat_with_init_messages = Chat(model="gpt-4o-mini",
    init_messages=[{"role": "system", "content": "You are an unhelpful assistant. Whenever asked to help, you say no."}],
)
res = await chat_with_init_messages(messages=[{"role": "user", "content": "What is the capital of {{country}}?"}],country="France")
res
assert 'no' in res['content'].lower(),res

In [ ]:
chat_with_init_messages_anthropic = Chat(model="claude-sonnet-4-5",
    init_messages=[{"role": "system", "content": "You are an unhelpful assistant. Whenever asked to help, you say no."}],
)
res = await chat_with_init_messages_anthropic(messages=[{"role": "user", "content": "What is the capital of {{country}}?"}],country="France")
res
assert 'no' in res['content'].lower(),res

## Image to Text

In [ ]:
#| export
@disk_cache.cache
async def image_to_text(path:str,model:str="gpt-4o-mini",url=False):
    """
    This function takes an image (either from a local file path or URL) and uses OpenAI's
    vision model to generate a detailed description of the image contents. The results are
    cached using disk_cache to avoid redundant API calls.
        
    Args:
        path (str): Path to the image file or URL of the image
        model (str, optional): OpenAI model to use for image analysis. Defaults to "gpt-4o-mini".
        url (bool, optional): Whether the path is a URL. Defaults to False.
        
    Returns:
        dict: A dictionary containing:
            - role (str): Always "assistant"
            - content (str): Detailed description of the image
            - meta (dict): Usage statistics including input and output tokens
    
    """
    if url:
        image = instructor.Image.from_url(path)
    else:
        image = instructor.Image.from_path(path)

    class ImageAnalyzer(BaseModel):
        description:str

    res,usage = await complete(
        model=model,
        messages=[{"role":"user","content":[
            "What is in this image, please describe it in detail\n",
            image,
            "\n"
        ]}],
        response_model=ImageAnalyzer,
    )
    return {
        'role':'assistant',
        'content':res.description,
        'meta':usage
    }


In [ ]:
from textwrap import wrap

In [ ]:
res= await image_to_text(get_git_root()/"sample_data/fox.jpeg")

assert 'fox' in res['content']
print('\n'.join(wrap(res['content'],width=100)))
res

The image features a close-up of a fox's face, showcasing its distinct features. The fox has a bushy
coat with a mix of reddish-brown and cream colors, with a characteristic white patch on its chin.
Its ears are pointed and alert, standing upright, with a lighter fur lining. The eyes are sharp and
focused, exhibiting an amber hue that contrasts with its fur. The background appears softly blurred,
adding emphasis to the fox's facial details and enhancing the warm tones of its fur.


{'role': 'assistant',
 'content': "The image features a close-up of a fox's face, showcasing its distinct features. The fox has a bushy coat with a mix of reddish-brown and cream colors, with a characteristic white patch on its chin. Its ears are pointed and alert, standing upright, with a lighter fur lining. The eyes are sharp and focused, exhibiting an amber hue that contrasts with its fur. The background appears softly blurred, adding emphasis to the fox's facial details and enhancing the warm tones of its fur.",
 'meta': {'input_tokens': 8627, 'output_tokens': 107}}

In [ ]:
res= await image_to_text(path = get_git_root()/"sample_data/fox.jpeg", model = "claude-sonnet-4-5")

assert 'fox' in res['content']
print('\n'.join(wrap(res['content'],width=100)))
res


This is a close-up portrait photograph of a red fox (Vulpes vulpes) against a soft, blurred gray-
beige background. The fox is captured in sharp detail, showing its distinctive features: pointed
triangular ears with black backs and white/cream interiors, a reddish-orange coat on the face and
head, and a white/cream-colored chest and throat area. The fox has striking amber or golden-colored
eyes with dark pupils, a black nose, and delicate white whiskers. The fur appears soft and fluffy,
with the characteristic russet coloring that gives the red fox its name. The fox's expression
appears calm and alert, gazing slightly off to the side. The lighting is natural and soft,
highlighting the texture of the fur and creating a warm, intimate portrait. The composition focuses
on the fox's face and upper body, with the background intentionally out of focus to emphasize the
subject.


{'role': 'assistant',
 'content': "This is a close-up portrait photograph of a red fox (Vulpes vulpes) against a soft, blurred gray-beige background. The fox is captured in sharp detail, showing its distinctive features: pointed triangular ears with black backs and white/cream interiors, a reddish-orange coat on the face and head, and a white/cream-colored chest and throat area. The fox has striking amber or golden-colored eyes with dark pupils, a black nose, and delicate white whiskers. The fur appears soft and fluffy, with the characteristic russet coloring that gives the red fox its name. The fox's expression appears calm and alert, gazing slightly off to the side. The lighting is natural and soft, highlighting the texture of the fur and creating a warm, intimate portrait. The composition focuses on the fox's face and upper body, with the background intentionally out of focus to emphasize the subject.",
 'meta': {'input_tokens': 194, 'output_tokens': 212}}

In [ ]:
from textwrap import wrap


In [ ]:
image_url = 'https://www.boredpanda.com/blog/wp-content/uploads/2020/07/funny-expressive-dog-corgi-genthecorgi-1-1-5f0ea719ea38a__700.jpg?utm_campaign=rebelboost_true'
res= await image_to_text(image_url,url=True)

assert 'corgi' in res['content']
print('\n'.join(wrap(res['content'],width=100)))
res


The image features a corgi dog with a playful expression. It is leaning over a wooden table,
propping itself up with its front paws, which are visible on the table. The dog's large, round eyes
are wide and focused on a slice of pizza on a white plate in front of it. The corgi has a slightly
open mouth with its tongue sticking out, adding a humorous touch to its eager demeanor. The pizza
appears to have various toppings, including what looks like pepperoni and vegetables. The background
is blurred, emphasizing the corgi and the pizza.


{'role': 'assistant',
 'content': "The image features a corgi dog with a playful expression. It is leaning over a wooden table, propping itself up with its front paws, which are visible on the table. The dog's large, round eyes are wide and focused on a slice of pizza on a white plate in front of it. The corgi has a slightly open mouth with its tongue sticking out, adding a humorous touch to its eager demeanor. The pizza appears to have various toppings, including what looks like pepperoni and vegetables. The background is blurred, emphasizing the corgi and the pizza.",
 'meta': {'input_tokens': 25628, 'output_tokens': 120}}

In [ ]:
image_url = 'https://www.boredpanda.com/blog/wp-content/uploads/2020/07/funny-expressive-dog-corgi-genthecorgi-1-1-5f0ea719ea38a__700.jpg?utm_campaign=rebelboost_true'
res= await image_to_text(image_url,url=True, model = "claude-sonnet-4-5")

assert 'corgi' in res['content']
print('\n'.join(wrap(res['content'],width=100)))
res


This is an adorable and humorous photograph of a Pembroke Welsh Corgi sitting at a wooden table,
appearing eager and excited about the food in front of it. The dog has the characteristic tan/orange
and white coloring typical of the breed, with large upright pointed ears, dark expressive eyes, and
a black nose. The corgi's pink tongue is sticking out slightly, and both front paws are raised up
and resting on the edge of the table in an endearing begging position. The dog's facial expression
shows clear interest and anticipation. On a white plate in front of the dog appears to be a serving
of pizza or some kind of baked dish with melted cheese and toppings. The background is softly
blurred, showing what seems to be an indoor setting with neutral-colored walls or surfaces. The
overall composition creates a charming and relatable scene that captures the universal experience of
dogs begging for human food, with the corgi's expressive face and body language conveying pure
excitement and hope

{'role': 'assistant',
 'content': "This is an adorable and humorous photograph of a Pembroke Welsh Corgi sitting at a wooden table, appearing eager and excited about the food in front of it. The dog has the characteristic tan/orange and white coloring typical of the breed, with large upright pointed ears, dark expressive eyes, and a black nose. The corgi's pink tongue is sticking out slightly, and both front paws are raised up and resting on the edge of the table in an endearing begging position. The dog's facial expression shows clear interest and anticipation. On a white plate in front of the dog appears to be a serving of pizza or some kind of baked dish with melted cheese and toppings. The background is softly blurred, showing what seems to be an indoor setting with neutral-colored walls or surfaces. The overall composition creates a charming and relatable scene that captures the universal experience of dogs begging for human food, with the corgi's expressive face and body language

## Speech to text

In [ ]:
#| export
from instructor.multimodal import Audio
import openai

In [ ]:
#| export

@disk_cache.cache
async def speech_to_text(audio_path: str, model: str = "whisper-1") -> Dict[str, str]:
    """Extract text from an audio file using OpenAI's Whisper model.
    
    Args:
        audio_path (str): Path to the audio file
        model (str, optional): OpenAI model to use. Defaults to "whisper-1".
    
    Returns:
        dict: A dictionary containing:  
            - role (str): Always "assistant"
            - content (str): Transcribed text from the audio
    """
    client = raw_openai_client()

    with open(audio_path, "rb") as audio_file:
        response = await client.audio.transcriptions.create(
            model=model,
            file=audio_file
        )
    
    res = {
        'role': 'assistant',
        'content': response.text,
    }
    
    return res

In [ ]:
res = await speech_to_text(get_git_root()/"sample_data/happy_speech.wav")
assert res['content'] == "Look at this, my hands are standing up in my arms, I'm giving myself goosebumps." , res
res

{'role': 'assistant',
 'content': "Look at this, my hands are standing up in my arms, I'm giving myself goosebumps."}

## Export

In [ ]:
# |hide
import nbdev

nbdev.nbdev_export()